In [12]:
import numpy as np
import os
import pandas as pd
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [13]:
import sqlite3 as sql

In [14]:
dirWork         = os.getcwd()
dirInput        = os.path.join(dirWork, r'input'       )
dirIntermediate = os.path.join(dirWork, r'intermediate')
dirResults      = os.path.join(dirWork, r'results'     )

print("Working Directory: "      + dirWork        )
print("Input Directory: "        + dirInput       )
print("Intermediate Directory: " + dirIntermediate)
print("Results Directory: "      + dirResults     )

Working Directory: E:\GitHub\UDOT-Segment-Factors
Input Directory: E:\GitHub\UDOT-Segment-Factors\input
Intermediate Directory: E:\GitHub\UDOT-Segment-Factors\intermediate
Results Directory: E:\GitHub\UDOT-Segment-Factors\results


In [15]:
df_CCSHourDirLane_Avenue = pd.read_csv(os.path.join(dirIntermediate,'CCSHourDirLane_Avenue.csv'))
display(df_CCSHourDirLane_Avenue)

df_CCSHourDirLane_OldFormat = pd.read_csv(os.path.join(dirIntermediate,'CCSHourDirLane_2019Aug.csv'))
display(df_CCSHourDirLane_OldFormat)

df_CCSHourDirLane_NewFormat = pd.read_csv(os.path.join(dirIntermediate,'CCSHourDirLane_NewFormat.csv'))
display(df_CCSHourDirLane_NewFormat)

,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2013,1,1,0,0,358,1
1,301,N,2013,1,1,0,1,435,1
2,301,N,2013,1,1,0,2,277,1
3,301,N,2013,1,1,0,3,163,1
4,301,N,2013,1,1,0,4,129,1
...,...,...,...,...,...,...,...,...,...
12281339,714,P,2018,9,30,1,19,41,6
12281340,714,P,2018,9,30,1,20,41,6
12281341,714,P,2018,9,30,1,21,20,6
12281342,714,P,2018,9,30,1,22,20,6


,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2019,8,11,1,0,511,6
1,301,N,2019,8,11,1,1,284,6
2,301,N,2019,8,11,1,2,188,6
3,301,N,2019,8,11,1,3,140,6
4,301,N,2019,8,11,1,4,147,6
...,...,...,...,...,...,...,...,...,...
269978,714,P,2019,8,31,1,17,68,5
269979,714,P,2019,8,31,1,18,38,5
269980,714,P,2019,8,31,1,20,36,5
269981,714,P,2019,8,31,1,22,23,5


,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2018,1,1,1,0,155,0
1,301,N,2018,1,1,2,0,238,0
2,301,N,2018,1,1,3,0,82,0
3,301,P,2018,1,1,1,0,80,0
4,301,P,2018,1,1,2,0,167,0
...,...,...,...,...,...,...,...,...,...
8357487,716,N,2019,7,29,2,0,62,0
8357488,716,N,2019,7,29,3,0,112,0
8357489,716,P,2019,7,29,1,0,3,0
8357490,716,P,2019,7,29,2,0,42,0


In [16]:
# initialize dataframe with avenue data
df_CCSHourDirLane = df_CCSHourDirLane_Avenue.copy()

#add 2019 Aug and no august data
df_CCSHourDirLane = df_CCSHourDirLane.append(df_CCSHourDirLane_OldFormat, ignore_index=True)
df_CCSHourDirLane = df_CCSHourDirLane.append(df_CCSHourDirLane_NewFormat, ignore_index=True)

df_CCSHourDirLane

,STATION,DIR,YEAR,MONTH,DAY,LANE,HOUR,HOURVOL,DOW
0,301,N,2013,1,1,0,0,358,1
1,301,N,2013,1,1,0,1,435,1
2,301,N,2013,1,1,0,2,277,1
3,301,N,2013,1,1,0,3,163,1
4,301,N,2013,1,1,0,4,129,1
...,...,...,...,...,...,...,...,...,...
20908814,716,N,2019,7,29,2,0,62,0
20908815,716,N,2019,7,29,3,0,112,0
20908816,716,P,2019,7,29,1,0,3,0
20908817,716,P,2019,7,29,2,0,42,0


In [17]:
df_CCSHourDirLane.set_index(['STATION','DIR','YEAR','MONTH','DAY','LANE','HOUR'])

HOURVOL  DOW
STATION DIR YEAR MONTH DAY LANE HOUR              
301     N   2013 1     1   0    0         358    1
                                1         435    1
                                2         277    1
                                3         163    1
                                4         129    1
...                                       ...  ...
716     N   2019 7     29  2    0          62    0
                           3    0         112    0
        P   2019 7     29  1    0           3    0
                           2    0          42    0
                           3    0          49    0

[20908819 rows x 2 columns]

In [18]:
df_CCSHourDir = df_CCSHourDirLane.groupby(['STATION','YEAR','MONTH','DAY','DOW','HOUR','DIR'],as_index=False).agg({'HOURVOL':[np.sum]})
df_CCSHourDir = df_CCSHourDir.droplevel(1, axis=1)
df_CCSHourDir

,STATION,YEAR,MONTH,DAY,DOW,HOUR,DIR,HOURVOL
0,301,2013,1,1,1,0,N,358
1,301,2013,1,1,1,0,P,288
2,301,2013,1,1,1,1,N,435
3,301,2013,1,1,1,1,P,292
4,301,2013,1,1,1,2,N,277
...,...,...,...,...,...,...,...,...
10707698,716,2019,6,29,5,0,P,7494
10707699,716,2019,6,30,6,0,N,5032
10707700,716,2019,6,30,6,0,P,4707
10707701,716,2019,7,29,0,0,N,7037


In [19]:
df_CCSHourDir.dtypes

STATION     int64
YEAR        int64
MONTH       int64
DAY         int64
DOW         int64
HOUR        int64
DIR        object
HOURVOL     int64
dtype: object

In [21]:
#for checks and understanding
#calculate number of days with data per station per month
df_1 = df_CCSHourDir.groupby(['STATION','MONTH','YEAR','DAY'],as_index=False).agg({'HOURVOL':[np.size]})
df_2 = df_1.groupby(['STATION','YEAR','MONTH'],as_index=False).agg({'DAY':[np.size]})
df_2 = df_2.droplevel(1, axis=1)
df_3 = df_2.pivot_table(index=['STATION'], columns=('YEAR','MONTH'), values='DAY').reset_index()

df_3 = df_3.fillna(0)
df_3 = df_3.astype(np.int32)

#export
df_3.to_csv(os.path.join(dirIntermediate, r'StationMonthDays.csv'),index=False)

In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_3)
pd.set_option('display.max_rows', 10)

YEAR  STATION 2013                                             2014          \
MONTH            1   2   3   4   5   6   7   8   9  10  11  12    1   2   3   
0         301   31  28  30  30  31  28  28  31  30  22  30  31   31  28  30   
1         302   31  28  30  30  31  30  31  29  30  31  30  31   31  28  30   
2         303   23  27  30  30  31  30  31  31  30  31  30  31   31  28  30   
3         304   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
4         305   28  28  30  30  28  30  25  31  30  31  30  31   31  22  30   
5         306   30  25  30  30  31  30  31  31  30  31  30  31   31  28  30   
6         307   31  28  30  30  31  30  31  31  30  31  30  27   31  20  30   
7         308   31  28  30  30  31  30  30  31  30  31  29  28   31  28  30   
8         309   25  28  30  30  31  30  30  31  30  31  30  27   31  28  29   
9         310   31  27  30  30  30  22  31  31  30  31  30  31   31  28  29   
10        312   23  28  28  30  18  30  31  31  30  31  30  31   30  28  29   
11        313   31  28  30  30  31  30  31  31  30  31  30  29   31  28  29   
12        314   30  28  30  30  31  30  31  30  30  31  11  31   31  28  30   
13        315   30  28  29  30  31  29  31  31  30  31  30  31   31  28  30   
14        316   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
15        317   31  28  30  30  31  30  31  31  30  31  30  29   31  28  30   
16        318   27  28  30  30  31  30  31  31  30  31  30  26   31  28  29   
17        319   31  28  29  30  31  30  31  31  30  31  30  31   31  28  30   
18        320   24  28  30  30  22  18  27  30  26  29  30  28   31  25  30   
19        321   31  28  30  30  31  30  31  31  30  31  30  31   31  28  29   
20        322   31  28  30  30  31  30  27  30  30  25  30  31   31  28  30   
21        323   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
22        324   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
23        325   31  28  30  30  30  30  31  31  30  31  30  31   31  28  30   
24        327   31  28  29  30  31  30  31  31  30  31  30  31   30  28  30   
25        329   22  28  30  30  31  30  31  31  30  31  30  25   29  24  29   
26        332   31  25  30  30  22  30  31  31  30  15  23  27   19  28  26   
27        333   31  28  30  30  31  30  31  31  29  31  30  30   30  28  30   
28        335   31  28  30  30  31  30  29  28  30  27  30  29   30  28  30   
29        340   31  26  28  30  31  30  31  31  30  31  30  31   31  28  30   
30        341   31  28  30  30  31  29  30  31  30  31  30  31   31  28  29   
31        348   23  28  30  30  31  30  31  31  30  31  28  25   31  28  30   
32        349   31  28  30  30  30  30  31  31  30  31  29  31   31  28  30   
33        350   26  28  30  30  31  30  31  31  30  31  30  29   31  28  29   
34        351   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
35        353   31  28  30  30  30  30  31  31  30  31  30  30   31  28  29   
36        354   31  28  30  30  31  30  31  31  29  31  30  30   31  28  30   
37        355   31  28  30  30  31  30  31  31  30  31  30  31   31  18  30   
38        362   31  28  30  30  31  30  31  31  30  31  30  31   31  28  30   
39        363   31  28  30  30  31  30  31  27  30  31  30  31   31  28  29   
40        382   31  28  30  30  31  30  31  31  26  31  30  31   31  24  26   
41        400   31  26  29  30  30  30  31  31  28  29  29  31   31  28  27   
42        401   31  28  30  30  31  30  31  31  30  28  28  25   12  27  29   
43        402   31  28  30  30  31  30  31  31  29  30  26  27   29  24  29   
44        403   31  27  30  27  31  30  31  31  26  31  29  30   31  28  28   
45        404   31  28  30  30  31  30  31  27  27  31  30  31   31  28  31   
46        405   31  28  30  30  31  30  31  31  30  31  30  31   31  23  21   
47        406   31  28  30  30  31  30  31  31  30  31  30  31   31  28  29   
48        407   31  28  30  25  31  30  31  31  30

In [25]:
#export to csv
df_CCSHourDir.to_csv(os.path.join(dirIntermediate, r'CCSHourDir_Combined_2013to2019.csv'),index=False)